In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
test_data = pd.read_pickle("test_data_Lenskit2")
all_recs_joined = pd.read_pickle("all_recs_joined_Lenskit2")

In [5]:
def evaluate_genre(genre): 
    
    providers = list(all_recs_joined.iloc[:,8:])
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    rla.add_metric("APCR")
    rla.add_metric("nd_APCR")
    rla.add_metric("equal_ex")
    rla.add_metric("ndcg")
    results= rla.compute(all_recs_joined, test_data, genre, providers)
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [6]:
testproviders = ["Drama"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

Drama


C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


done Drama


In [7]:
output = pd.concat(allResults, keys=testproviders)
output

rND       rKL       rRD  APCR   nd_APCR  equal_ex  \
      Algorithm                                                           
Drama ALS        0.051199  0.004966  0.068568   1.0  0.961838  0.995894   
      FunkSVD    0.053351  0.003831  0.085831   1.0  0.959206  0.989653   
      ItemItem   0.052188  0.005619  0.072413   1.0  0.961785  0.987928   
      UserUser   0.065563  0.007243  0.102991   1.0  0.962698  0.981984   

                     ndcg  
      Algorithm            
Drama ALS        0.312561  
      FunkSVD    0.271353  
      ItemItem   0.288700  
      UserUser   0.266556

In [8]:
output.to_csv('out_1.csv')